# Clean and Analyze Employee Exit Surveys

## Introduction

dataset
- exit surveys of employees from Queensland, Australia
    - Department of Education, Training and Employment (DETE)
    - Technical and Further Education (TAFE)
    - encoded to UTF-8

project goal
- Are employes who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction?
- What about the employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction?
- What about older employees?

- combine results for both surveys to answer the quetions
- use same survey template, but one customized some of the answers
- no data dictionary available

skills:
- apply(), map()
- fillna(), dropna(), drop()
- melt()
- concat(), merge()

In [1]:
import numpy as np
import pandas as pd

## 1. The DETE and TAFE Survey Datasets

`dete_survey.csv`
* `ID` participant ID
* `SeparationType` reason why employment ended
* `Cease Date` year or month employment ended
* `DETE Start Date` year employemnt started

`tafe_survey.csv`
* `Record ID` participant ID
* `Reason for ceasing employment`
* `LengthofServiceOverall. Overall Length of Service at Institute (in years)` employment in years

In [2]:
# read in and preview datasets
dete_raw = pd.read_csv('/Users/slp22/code/dataquest projects/dete_survey.csv')
tafe_raw = pd.read_csv('/Users/slp22/code/dataquest projects/tafe_survey.csv')

### DETE

In [3]:
# dete_raw.head()

In [4]:
# dete_raw.info()

In [5]:
# dete_raw.columns

In [6]:
# dete_raw.isnull()

In [7]:
# dete_raw['SeparationType'].value_counts()

In [8]:
# dete_raw['Position'].value_counts()

In [9]:
# dete_raw['Classification'].value_counts()

**`dete_raw`**
- RangeIndex: 822 entries, 0 to 821
- Data columns (total 56 columns)
- Dytpe: ID=int, others=object, bool
- Non-Null: Business Unit, Aboriginal, Torres Strait, South Sea, Disability, NESB

### TAFE

In [10]:
# tafe_raw.head()

In [11]:
# tafe_raw.info()

In [12]:
# tafe_raw.columns

In [13]:
# tafe_raw.isnull()

In [14]:
# tafe_raw['Reason for ceasing employment'].value_counts()

In [15]:
# tafe_raw['Employment Type. Employment Type'].value_counts()

In [16]:
# tafe_raw['Classification. Classification'].value_counts()

**`tafe_raw`**
- Record ID in scientific notation
- Columns names are long, descriptive, repetitive
- RangeIndex: 702 entries, 0 to 701
- Data columns (total 72 columns)
- Dtype: ID=int, others=object, cessation year=float
- Non-Null: range 400-500 of 700 rows

## 2. Identify Missing Values and Drop Unnecessary Columns

In [17]:
# dete_raw = pd.read_csv('/dete_survey.csv', na_values="Not Stated")
dete_raw = pd.read_csv('/Users/slp22/code/dataquest projects/dete_survey.csv')

In [18]:
#dete_raw drop columns [28:49] axis=1
dete = dete_raw.drop(dete_raw.columns[28:49],axis=1)
# dete.head()

In [19]:
#tafe drop columns [17:66] axis=1
tafe = tafe_raw.drop(tafe_raw.columns[17:66], axis=1)
# tafe.head()

### Dropped columns from `tafe` [28:39] and `dete` [17:66] that are not relevant to this analysis. And will make the data easier to work with.  

## 3. Clean Column Names

In [20]:
dete_col = dete.columns
# dete_col

In [21]:
tafe_col = tafe.columns
# tafe_col

### 🧹 functions to clean up text

In [22]:
# function to make each column name lowercase
def lower(cols):
    lower_cols = []
    for c in cols:
        lower_cols.append(c.lower())
    return lower_cols

In [23]:
# function to remove trailing whitespace from end of strings
def spaceless(cols):
    spaceless_cols = []
    for c in cols:
        spaceless_cols.append(c.rstrip())
    return spaceless_cols

In [24]:
# function to replace space with underscore
def replace_punctuation(cols):
    underscore_cols = []
    for c in cols:
        new_c = c.replace(" ", "_").replace(".", "").replace("-", "")
        underscore_cols.append(new_c)
    return underscore_cols

### 🚫 apply clean up functions #1 (nested functions)

In [25]:
# from types import new_class
def clean_up(col):
    new_cols = []

# function to make each column name lowercase

#   # function to remove trailing whitespace from end of strings
#     def spaceless(new_cols):
#         for c in new_cols:
#             new_cols.append(c.rstrip())
#         return new_cols

#   # function to replace space with underscore
#     def replace_punctuation(new_cols):
#         for c in new_cols:
#             new_c = c.replace(" ", "_").replace(".", "").replace("-", "")
#             new_cols.append(new_c)
#         return new_cols
    # return lower(new_cols


In [26]:
# # higer order function lesson

# def generate_age_checker(min_age):
#     def check_age(age):
#         return age > min_age
#     return check_age

# check_min_18 = generate_age_checker(18)
# check_min_21 = generate_age_checker(21)

# print(check_min_18(20))
# print(check_min_21(20))
    

### ✅ apply clean up functions #2 (sequential)



In [27]:
# # appy lower, spaceless, and replace_punctuation functions for tafe_col
# lower_tafe = lower(tafe_col)
# spaceless_tafe = spaceless(lower_tafe)
# clean_tafe_cols = replace_punctuation(spaceless_tafe)

In [28]:
# # appy lower, spaceless, and replace_punctuation functions for dete_col
# lower_dete = lower(dete_col)
# spaceless_dete = spaceless(lower_dete)
# clean_dete_cols = replace_punctuation(spaceless_dete)

### ✅ apply clean up functions #3 (nest func)

In [29]:
# replace_punctuation(spaceless(lower(dete_col)))

In [30]:
# replace_punctuation(spaceless(lower(tafe_col)))

### ✅ apply clean up functions #4 (call func)

In [31]:
# best practice
def clean_up(col):
    lowercased = lower(col)
    without_spaces = spaceless(lowercased)
    without_punctuation = replace_punctuation(without_spaces)
    return without_punctuation

In [32]:
new_dete_col = clean_up(dete_col)
dete.columns = new_dete_col
dete.columns

Index(['id', 'separationtype', 'cease_date', 'dete_start_date',
       'role_start_date', 'position', 'classification', 'region',
       'business_unit', 'employment_status', 'career_move_to_public_sector',
       'career_move_to_private_sector', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload',
       'none_of_the_above', 'gender', 'age', 'aboriginal', 'torres_strait',
       'south_sea', 'disability', 'nesb'],
      dtype='object')

In [33]:
# tafe.columns

In [34]:
tafe.rename({'Record ID': 'id',
             'CESSATION YEAR': 'cease_date',
             'Reason for ceasing employment': 'separationtype',
             'Gender. What is your Gender?': 'gender',
             'CurrentAge. Current Age': 'age',
             'Employment Type. Employment Type': 'employment_status',
             'Classification. Classification': 'position',
             'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service',
             'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'}, 
            axis='columns',
           inplace=True)

In [35]:
# tafe.head()

### Renamed col names to make it easier to call. 

## 4. Filter the Data

*Filter data to answer*
- Are employees who have only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? 
- What about employees who have been at the job longer?

In [36]:
dete['separationtype'].unique() #'Resignation-Other reasons'
                                #'Resignation-Other employer'
                                #'Resignation-Move overseas/interstate'

array(['Ill Health Retirement', 'Voluntary Early Retirement (VER)',
       'Resignation-Other reasons', 'Age Retirement',
       'Resignation-Other employer',
       'Resignation-Move overseas/interstate', 'Other',
       'Contract Expired', 'Termination'], dtype=object)

In [37]:
tafe['separationtype'].unique() #'Resignation'

array(['Contract Expired', 'Retirement', 'Resignation',
       'Retrenchment/ Redundancy', 'Termination', 'Transfer', nan],
      dtype=object)

### `separationtype`

- `dete`
    - `Resignation-Other reasons`
    - `Resignation-Other employer`
    - `Resignation-Move overseas/interstate`
- `tafe`
    - `Resignation`

In [38]:
# #copy datasets
# dete.copy()
# tafe.copy()

print('dete.shape', dete.shape)
print('tafe.shape', tafe.shape)

dete.shape (822, 35)
tafe.shape (702, 23)


In [39]:
# Filter for resignation types
dete_resignation = dete.loc[(dete['separationtype'] == 'Resignation-Other reasons') | 
                             (dete['separationtype'] =='Resignation-Other employer') | 
                             (dete['separationtype'] == 'Resignation-Move overseas/interstate')]

In [40]:
# Filter for resignation
tafe_resignation = tafe[tafe['separationtype'] == 'Resignation']

In [41]:
print('dete_resignation.shape', dete_resignation.shape)
print('tafe_resignation.shape', tafe_resignation.shape)

dete_resignation.shape (311, 35)
tafe_resignation.shape (340, 23)


### Filtered for resignations only to answer question; filtered out 50% irrelevant data. 

## 5. Verify the Data

In [42]:
# function to clean up date columns by extracting the year from MM/YYYY or replacing 'Not Stated' with 0. 
def extract_year(cease_date):
    years = []
    for i in cease_date:
        if i == 'Not Stated':
            years.append(0) 
        elif len(i) == 7:
            years.append(int(i[3:7]))
        else:
            years.append(int(i))

    return years

### Dete Resignation Years

In [43]:
# before `cease_date`
dete_resignation['cease_date'].value_counts().sort_index(ascending=True)

01/2014        22
05/2012         2
05/2013         2
06/2013        14
07/2006         1
07/2012         1
07/2013         9
08/2013         4
09/2010         1
09/2013        11
10/2013         6
11/2013         9
12/2013        17
2010            1
2012          126
2013           74
Not Stated     11
Name: cease_date, dtype: int64

In [44]:
# run `cease_date` through extract_year() function
extract_year(dete_resignation['cease_date'])

[2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2012,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2013,
 2013,
 2013,
 2012,
 2012,
 2012,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2012,
 2012,
 2012,
 2013,
 2013,
 2013,
 2013,
 2012,
 2013,
 2013,
 2012,
 2012,
 2012,
 2012,
 2012,
 2012,
 2013,
 2012,
 2013,
 2013,
 2013,
 2012,
 2012,
 2013,
 2012,
 2013,

In [45]:
dete_resignation['cease_date'] = extract_year(dete_resignation['cease_date'])

/var/folders/m7/q9jqt2zs3b1b4t81n_tpn1lh0000gn/T/ipykernel_74363/2989179497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dete_resignation['cease_date'] = extract_year(dete_resignation['cease_date'])


In [46]:
# after `cease_date`
dete_resignation['cease_date'].value_counts().sort_index(ascending=True)

0        11
2006      1
2010      2
2012    129
2013    146
2014     22
Name: cease_date, dtype: int64

In [47]:
# before `dete_start_date`
dete_resignation['dete_start_date'].value_counts().sort_index(ascending=True)

1963           1
1971           1
1972           1
1973           1
1974           2
1975           1
1976           2
1977           1
1980           5
1982           1
1983           2
1984           1
1985           3
1986           3
1987           1
1988           4
1989           4
1990           5
1991           4
1992           6
1993           5
1994           6
1995           4
1996           6
1997           5
1998           6
1999           8
2000           9
2001           3
2002           6
2003           6
2004          14
2005          15
2006          13
2007          21
2008          22
2009          13
2010          17
2011          24
2012          21
2013          10
Not Stated    28
Name: dete_start_date, dtype: int64

In [48]:
# run `dete_start_date` through extract_year() function
extract_year(dete_resignation['dete_start_date'])

[2005,
 1994,
 2009,
 1997,
 2009,
 1998,
 2007,
 0,
 1982,
 1980,
 1997,
 1973,
 1995,
 2005,
 2003,
 2006,
 2011,
 0,
 1977,
 1974,
 2011,
 1976,
 2009,
 2009,
 1993,
 2008,
 2003,
 2011,
 2006,
 2011,
 2007,
 1986,
 2002,
 2011,
 2006,
 2002,
 2004,
 0,
 2008,
 2004,
 2007,
 1997,
 1976,
 2010,
 2012,
 1980,
 2012,
 2007,
 1994,
 2004,
 0,
 2007,
 2003,
 2011,
 2003,
 2005,
 2012,
 1998,
 2005,
 2006,
 1995,
 1989,
 0,
 2005,
 2008,
 2006,
 2007,
 1986,
 1999,
 1996,
 2009,
 1994,
 2009,
 2007,
 2011,
 2006,
 2000,
 2008,
 2005,
 2012,
 2007,
 2008,
 2011,
 2009,
 2011,
 2011,
 2011,
 2010,
 1991,
 2011,
 1992,
 2007,
 2007,
 2012,
 2012,
 0,
 0,
 0,
 1980,
 2006,
 1996,
 0,
 2005,
 2009,
 2001,
 1999,
 2001,
 1989,
 2012,
 2011,
 2007,
 2011,
 2000,
 2008,
 2011,
 2012,
 2005,
 0,
 1988,
 2008,
 1980,
 2007,
 1992,
 2003,
 0,
 2010,
 2012,
 2012,
 1992,
 1999,
 2007,
 1990,
 0,
 2008,
 1994,
 2007,
 1985,
 2000,
 2007,
 1993,
 0,
 0,
 1991,
 0,
 2006,
 0,
 2008,
 2008,
 1997,
 2011

In [49]:
dete_resignation['dete_start_date'] = extract_year(dete_resignation['dete_start_date'])

/var/folders/m7/q9jqt2zs3b1b4t81n_tpn1lh0000gn/T/ipykernel_74363/2585752466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dete_resignation['dete_start_date'] = extract_year(dete_resignation['dete_start_date'])


In [50]:
# after `dete_start_date`
dete_resignation['dete_start_date'].value_counts().sort_index(ascending=True)

0       28
1963     1
1971     1
1972     1
1973     1
1974     2
1975     1
1976     2
1977     1
1980     5
1982     1
1983     2
1984     1
1985     3
1986     3
1987     1
1988     4
1989     4
1990     5
1991     4
1992     6
1993     5
1994     6
1995     4
1996     6
1997     5
1998     6
1999     8
2000     9
2001     3
2002     6
2003     6
2004    14
2005    15
2006    13
2007    21
2008    22
2009    13
2010    17
2011    24
2012    21
2013    10
Name: dete_start_date, dtype: int64

### Tafe Resignation Years

In [51]:
# `cease_date`
tafe_resignation['cease_date'].value_counts().sort_index(ascending=True)

2009.0      2
2010.0     68
2011.0    116
2012.0     94
2013.0     55
Name: cease_date, dtype: int64

### Observations:
- Two columns needed the date format cleaned up. 
- Others are in int and are only years. 
- None of the years are before 1940 or after 2014. 
- Limited number of years that span the two data sets. May need to limit query to 2012-2014.

## 6. Create a New Column

In [52]:
dete_resignation['institute_service'] = dete_resignation['cease_date'] - dete_resignation['dete_start_date']

# dete_resignation['dete_start_date'].info()

/var/folders/m7/q9jqt2zs3b1b4t81n_tpn1lh0000gn/T/ipykernel_74363/1908586038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dete_resignation['institute_service'] = dete_resignation['cease_date'] - dete_resignation['dete_start_date']


In [53]:
dete_resignation['institute_service']

3         7
5        18
8         3
9        15
11        3
       ... 
808       3
815       2
816       2
819       5
821    2013
Name: institute_service, Length: 311, dtype: int64

### To calculate the length of service (employment), substract start year from cease year. 

# 7. Identify Dissatisfied Employees